<p style="font-weight:bold; font-size: 40px; color:#0171ff; padding-top: 50px; text-align:center"> How to create a new Scenario </p>

The International Financial Reporting Standard (IFRS) 17 dictates companies to  

> "... (a) exposures to risks and how they arise;

> (b) objectives, policies and processes for managing risks and the methods used to measure those risks;

>(c) concentrations of risk arising from insurance contracts;

>(d) the claims development—ie actual claims compared with previous estimates of the undiscounted amount of the claims;

>(e) the credit quality of reinsurance contract assets; and

>(f) liquidity risk, including a maturity analysis showing the estimated cash flows arising from insurance contracts... [1]"

but it does not force or prescribe *how* each Scenario should be modeled and *how many* or them should be considered.

Systemorph IFRS17 Solution gives the User the possibility to interpretate the requirements with the highest flexibility possible thanks to its open source approach.

In this notebook we focus on the creation of new Scenario and new AoC step to host them. Given some simple rules, the solution automatically incorporate new arbitrary AoC steps and then calculate PVs, CSMs and FP, calculating the Deltas from cumulative cash flows as usual.

In [0]:
#!import "//develop-ifrs17/v1.2.0/CalculationEngine"

In [0]:
await DataSource.SetAsync();
DataSource.Reset(x => x.ResetCurrentPartitions());

In [0]:
await Import.FromFile("Dimensions.xlsx")
    .WithType<ReportingNode>()               .WithType<AocType>()                .WithType<DeferrableAmountType>()
    .WithType<AmountType>()                  .WithType<Scenario>()               .WithType<LiabilityType>()
    .WithType<LineOfBusiness>()              .WithType<EstimateType>()           .WithType<EconomicBasis>()
    .WithType<Currency>()                    .WithType<PnlVariableType>()        .WithType<BsVariableType>()
    .WithType<Novelty>()                     .WithType<Profitability>()          .WithType<OciType>()
    .WithType<ValuationApproach>()           .WithType<RiskDriver>()             .WithType<ProjectionConfiguration>()    
    .WithType<ExchangeRate>()
    .WithTarget(DataSource)
    .ExecuteAsync()

The AoC Configuration is a special dimension that contains the business rules to perform the analysis of change from cash flows and actuals. It also contains default values for new AoC steps. With them 

In [0]:
await Import.FromFile("Dimensions_EP3.xlsx")
            .WithFormat(ImportFormats.AocConfiguration)
            .WithTarget(DataSource)
            .ExecuteAsync()

In [0]:
await Import.FromFile("DataNodes.xlsx")
            .WithFormat(ImportFormats.DataNode)
            .WithTarget(DataSource)
            .ExecuteAsync()

In [0]:
await Import.FromFile("YieldCurve.xlsx").WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync()

In [0]:
var ifrs17 = new Ifrs17(Workspace, Scopes, Report);

In [0]:
await Import.FromFile("CF_G_2022_12.xlsx").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromFile("A_G_2022_12.csv").WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromFile("A_G_2023_1.csv").WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
Workspace.Reset(x => x.ResetInitializationRules());
Workspace.InitializeFrom(DataSource);

In [0]:
//await Import.FromFile("CF_G_2023_1.xlsx").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromFile("CF_G_2023_3.xlsx").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromFile("CF_G_2023_3_IIUUP.xlsx").WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
var aocTypes = await DataSource.Query<AocType>().OrderBy(x => x.Order).ToArrayAsync();
    var aocTypesCompulsory = typeof(AocTypes).GetFields().Select(x => (string)x.Name);
    if(aocTypesCompulsory.Where(x => !aocTypes.Select(x => x.SystemName).Contains(x)).Any()) {
        ApplicationMessage.Log(Error.AocTypeCompulsoryNotFound);
        return Activity.Finish();
    }
    
    
    var orderByName = aocTypes.ToDictionary(x => x.SystemName, x => x.Order);
    var aocConfigs = (await DataSource.Query<AocConfiguration>().ToArrayAsync())
                                     .GroupBy(x => (x.AocType, x.Novelty))
                                     .Select(y => y.OrderByDescending(x => x.Year).ThenByDescending(x => x.Month).FirstOrDefault())
                                     .ToDictionary(x => (x.AocType, x.Novelty));
    var aocOrder = aocConfigs.ToDictionary(x => x.Key, x => x.Value.Order);
    var newAocTypes = orderByName.Keys.Where(x => !aocConfigs.Keys.Contains((x, Novelties.I)) && 
                                                  !aocConfigs.Keys.Contains((x, Novelties.N)) && 
                                                  !aocConfigs.Keys.Contains((x, Novelties.C)) && 
                                                  !aocTypes.Any(y => y.Parent == x)           &&
                                                  !aocTypesCompulsory.Contains(x)).ToArray();

    aocOrder